# Preprocessing Data and Preparing Train and VAl for Pytorch

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from tqdm import tqdm

import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

In [2]:
tqdm.pandas()

In [3]:
train = pd.read_csv("data/toxic_competition_data/train.csv")
print(train.shape)
train.head()

(159571, 8)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
df = train.loc[(train.severe_toxic == 1) & (train.obscene == 1), :].reset_index(drop=True)
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
1,0020e7119b96eeeb,Stupid peace of shit stop deleting my stuff as...,1,1,1,0,1,0
2,006e87872c8b370c,you are a stupid fuck \n\nand your mother's cu...,1,1,1,0,1,0
3,01208d2b76624130,Hi \n\nIm a fucking bitch.\n\n50.180.208.181,1,1,1,0,1,0
4,018663f910e0bfe6,What a motherfucking piece of crap those fuckh...,1,1,1,0,1,0


In [5]:
# train = train[["comment_text", "toxic"]]

In [6]:
test = pd.read_csv("data/toxic_competition_data/test.csv")
print(test.shape)
test.head()

(153164, 2)


,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [7]:
train.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [8]:
test.isnull().sum()

id              0
comment_text    0
dtype: int64

In [9]:
replace_ip=re.compile(r'([0-9]+)(?:\.[0-9]+){3}',)
def text_to_wordlist(text, remove_stopwords=True, stem_words=False, replace_ip=replace_ip):
    # Clean the text, with the option to remove stopwords and to stem words.
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    #Replace IP address
    text=replace_ip.sub('',text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [10]:
train["comment_text"] = train.comment_text.progress_apply(text_to_wordlist)

100%|██████████| 159571/159571 [00:23<00:00, 6911.94it/s]


In [11]:
train.toxic.value_counts()

0    144277
1     15294
Name: toxic, dtype: int64

In [12]:
train.severe_toxic.value_counts()

0    157976
1      1595
Name: severe_toxic, dtype: int64

In [13]:
train, val = train_test_split(train, test_size=0.1, shuffle=True) #, stratify=train[['toxic']])
train.shape, val.shape

((143613, 8), (15958, 8))

In [14]:
train.to_csv("data/toxic_competition_data/train_torch.csv", encoding="utf-8", index=False)
val.to_csv("data/toxic_competition_data/test_torch.csv", encoding="utf-8", index=False)

In [15]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')